In [1]:
# Code that helps avoid overusing memory

import tensorflow as tf
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config)
from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


In [2]:
#To access gym
import sys
sys.path.append('/home/kaiolae/.virtualenvs/worldmodels/lib/python3.5/site-packages')

In [3]:
#Importing the VAE and RNN.
import os

#Adding WorldModels path to pythonpath
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
#Importing the VAE
from doomrnn import ConvVAE as VAE
from doomrnn import Model as RNN
from collections import namedtuple

/home/kaiolae/code/word_models_keras_test/WorldModels_hardmaru


In [5]:
LATENT_SPACE_DIMENSIONALITY = 64
NUM_MIXTURES = 5 #TODO Get from dir name
RNN_SIZE = 512
model_restart_factor = 10.

In [30]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML


def plot_movie_mp4(image_array):
    dpi = 2.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    #fig = plt.figure(figsize=(1,1), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [7]:
#RNN parameters - modelled after hps_sample in doomrnn.py
HyperParams = namedtuple('HyperParams', ['max_seq_len',
                                         'seq_width',
                                         'rnn_size',
                                         'batch_size',
                                         'grad_clip',
                                         'num_mixture',
                                         'restart_factor',
                                         'learning_rate',
                                         'decay_rate',
                                         'min_learning_rate',
                                         'use_layer_norm',
                                         'use_recurrent_dropout',
                                         'recurrent_dropout_prob',
                                         'use_input_dropout',
                                         'input_dropout_prob',
                                         'use_output_dropout',
                                         'output_dropout_prob',
                                         'is_training',
                                        ])

def default_prediction_hps():
  return HyperParams(max_seq_len=2, # KOEChange. Was 500
                     seq_width=LATENT_SPACE_DIMENSIONALITY,    # KOEChange. Was 32
                     rnn_size=RNN_SIZE,    # number of rnn cells
                     batch_size=1,   # minibatch sizes
                     grad_clip=1.0,
                     num_mixture=NUM_MIXTURES,   # number of mixtures in MDN
                     restart_factor=model_restart_factor, # factor of importance for restart=1 rare case for loss.
                     learning_rate=0.001,
                     decay_rate=0.99999,
                     min_learning_rate=0.00001,
                     use_layer_norm=0, # set this to 1 to get more stable results (less chance of NaN), but slower
                     use_recurrent_dropout=0,
                     recurrent_dropout_prob=0.90,
                     use_input_dropout=0,
                     input_dropout_prob=0.90,
                     use_output_dropout=0,
                     output_dropout_prob=0.90,
                     is_training=0)

In [8]:
#Loading Weights of trained VAE and RNN
rnn_model_path_name = "tf_rnn_5mixtures" #TODO Generalize
vae_path_name = "old_tf_models" #Keeping the 64-dim VAE here.

vae = VAE(z_size=LATENT_SPACE_DIMENSIONALITY,
              batch_size=1,
              is_training=False,
              reuse=False,
              gpu_mode=False)

vae.load_json(os.path.join(vae_path_name, 'vae.json'))

hps_sample = default_prediction_hps()
rnn = RNN(hps_sample, gpu_mode=False)
rnn.load_json(os.path.join(rnn_model_path_name, 'rnn.json'))

#untrained_rnn = RNN(decoder_mode=True)

INFO:tensorflow:Model using cpu.
model using cpu
input dropout mode = False
output dropout mode = False
recurrent dropout mode = False


In [23]:
#Get a start video
#Getting data to feed into the VAE and RNN
import numpy as np
# load preprocessed data
raw_data = np.load(os.path.join("series", "series.npz"))
raw_data_mu = raw_data["mu"]
raw_data_logvar = raw_data["logvar"]
raw_data_action =  raw_data["action"]

single_action_sequence = raw_data_action[16]
single_latent_sequence = raw_data_mu[16]
print("Actions length: ", len(single_action_sequence))
print("Latent vectors length: ", len(single_latent_sequence))
num_frames = len(single_latent_sequence)



#Decode all 100 vectors, store as video
#Visualize video.

Actions length:  177
Latent vectors length:  177


In [31]:
#Decode a sequence with the VAE and visualize it
def decode_and_visualize(latent_vector_sequence):
    reconstructions = vae.decode(np.array(latent_vector_sequence))
    print(reconstructions.shape)
    plot_movie_mp4(reconstructions)

### Visualizing one sequence of latent vectors from the training data

In [32]:
#From the actual sampled images. All loss here is due to VAE.
decode_and_visualize(single_latent_sequence)

(177, 64, 64, 3)


<Figure size 64x64 with 0 Axes>

## Predicting into the future with Ha's RNN model.

In [66]:
def get_pi_idx(x, pdf):
  # samples from a categorial distribution
  N = pdf.size
  accumulate = 0
  for i in range(0, N):
    accumulate += pdf[i]
    if (accumulate >= x):
      return i
  random_value = np.random.randint(N)
  #print('error with sampling ensemble, returning random', random_value)
  return random_value

from gym.utils import seeding
np_random, seed = seeding.np_random(None)

In [73]:
#Initializing these here. They should be updated each iteration.
rnn_state = rnn.sess.run(rnn.zero_state)
z = single_latent_sequence[0] #TODO REplace with random start frame.
restart = 0 #Just hardcoding for now.
dreamed_sequence = []


#Stepping through RNN. Modelled after _step in doomrnn.py.

for i in range(100):

    prev_z = np.zeros((1, 1, LATENT_SPACE_DIMENSIONALITY))
    prev_z[0][0] = z

    prev_action = np.zeros((1, 1))
    prev_action[0] = 0 #Just hardcoding for now.

    prev_restart = np.ones((1, 1))
    prev_restart[0] = restart 

    #Just hardcoding for now. Has huge effects. 1.0 gives wild dreams. 0.5 very boring.
    temperature = 1.0

    feed = {rnn.input_z: prev_z,
            rnn.input_action: prev_action,
            rnn.input_restart: prev_restart,
            rnn.initial_state: rnn_state
           }

    [logmix, mean, logstd, logrestart, next_state] = rnn.sess.run([rnn.out_logmix,
                                                                       rnn.out_mean,
                                                                       rnn.out_logstd,
                                                                       rnn.out_restart_logits,
                                                                       rnn.final_state], feed)
    OUTWIDTH = rnn.hps.seq_width

    # adjust temperatures
    logmix2 = np.copy(logmix)/temperature
    logmix2 -= logmix2.max()
    logmix2 = np.exp(logmix2)
    logmix2 /= logmix2.sum(axis=1).reshape(OUTWIDTH, 1)

    #The mixture models sampling?
    mixture_idx = np.zeros(OUTWIDTH)
    chosen_mean = np.zeros(OUTWIDTH)
    chosen_logstd = np.zeros(OUTWIDTH)
    for j in range(OUTWIDTH):
      idx = get_pi_idx(np_random.rand(), logmix2[j])
      mixture_idx[j] = idx
      chosen_mean[j] = mean[j][idx]
      chosen_logstd[j] = logstd[j][idx]

    rand_gaussian = np_random.randn(OUTWIDTH)*np.sqrt(temperature)
    next_z = chosen_mean+np.exp(chosen_logstd)*rand_gaussian #The next Z that I want
    print("Next z was: ", next_z)

    next_restart = 0
    done = False
    if (logrestart[0] > 0):
      next_restart = 1
      done = True

    z = next_z
    restart = next_restart
    rnn_state = next_state
    dreamed_sequence.append(z)

    reward = 1 # always return a reward of one if still alive.


Next z was:  [ 0.0735  1.2769  0.6551  1.4467  0.3146 -1.3128  1.2234  1.1155  0.2812 -0.7487  1.4803 -0.2774
  1.4727  0.4945 -0.7837 -0.9075  0.9218  0.8965 -0.1076  0.7058 -0.0082  0.0065 -0.5764 -0.3109
 -1.3491 -1.2537  0.5858 -0.155  -0.0051 -1.0774  1.9012 -0.0511 -0.6075 -1.4242  1.4611  1.0371
 -1.1872  2.4593  1.2838  0.3875 -1.0302 -2.2329  2.2367  0.6229  0.2473  0.419  -1.0384 -0.2149
 -0.7669 -0.5977  1.1231  1.1153  0.1955 -1.0656  0.0153 -0.1734  0.4281 -1.3049  0.7707  0.8607
 -0.2404  0.055  -0.3384  0.4849]
Next z was:  [-0.1957 -1.2017 -0.9917  0.8435  0.1072 -0.1415 -0.6323  0.6672  0.313  -0.268   0.1065  0.0817
 -1.1135  0.0114 -0.6851  0.2252 -1.1574  0.4371 -0.3763 -0.5913 -0.7877  0.9971 -0.6098  0.7222
 -0.858  -1.0524  0.6358 -1.282  -0.9798  0.0172  2.2052  0.0137 -0.8499 -1.1421  0.41   -1.1467
 -0.782   2.3868 -0.0464  1.2703 -1.576  -0.0328 -0.323   0.8184  1.5679  1.5464  0.2518  0.8291
 -2.7453 -0.3528  0.2241  1.6009 -0.7194  0.5136  0.6008  0.6936 -0

Next z was:  [ 0.4323 -0.4845 -0.0465  0.958   0.5332 -0.9647  0.2682 -1.697   1.3427 -0.1937 -0.2864  0.7556
  0.6519  0.8652 -0.5327  0.0779 -0.8396 -1.7801 -0.4869 -0.747   0.4468  0.0915  2.0443 -0.2699
 -0.7335  0.2368  0.4635  0.288   0.4911 -0.257  -0.3415  0.1748 -0.8096 -0.0285  2.025  -0.3547
 -0.6692  0.0389  0.7391 -1.6083 -0.2495  0.6571  1.464   0.1294 -0.1905  0.3288  0.9957  1.2114
 -0.7299  2.2282  0.2459  1.1228 -0.8647  0.8136  0.5832  1.0144 -0.7927  0.2866 -0.3328  2.4067
  0.0922 -2.8159 -0.8204  0.6148]
Next z was:  [ 0.4705 -1.8423 -0.4935 -0.7393  0.3503  0.1883 -0.654  -1.566   0.8731 -0.4027  1.426  -0.7183
 -0.3397  1.1603 -1.3633  0.1875  1.2734  0.8015  0.8367 -0.1301  0.9368 -1.4225  2.1079  0.4414
 -0.954   0.2945  0.7168  0.2981 -0.7076 -0.7404 -0.0082 -0.082  -0.2205 -0.1361  2.8032  1.913
 -0.6018  0.8659 -0.2184  0.7356  0.5958 -1.354  -0.7005  0.1096 -1.5292  0.2962 -0.3644  0.2132
 -0.3961 -0.3134  0.7921  1.6943 -0.8188 -1.2089  0.4009  1.5166  0.

In [74]:
#Visualizing the dream
decode_and_visualize(dreamed_sequence)

(100, 64, 64, 3)


<Figure size 64x64 with 0 Axes>

In [ ]:
#Decoding the sampled latent vectors into images, and visualizing
decode_and_visualize(single_latent_sequence)
#decode_and_visualize(generated_latent_vectors)
decode_and_visualize(untrained_latent_vectors)
#Visualizing the dream
decode_and_visualize(dreamed_latents)


In [ ]:
#How do I show current step and predicted next step nicely side-by-side?
TIMESTEP = 100
decoded_current = vae.decoder.predict(single_latent_sequence)
decoded_future = vae.decoder.predict(generated_latent_vectors)
#Decoding the exact input we gave at step TIMESTEP.
decoded_input_data = vae.decoder.predict(rnn_input_data[TIMESTEP][:,:-1])
print("Current image: ")
#The future is SEQ_SIZE timesteps ahead
plt.imshow(decoded_current[TIMESTEP+SEQ_SIZE]) 
plt.show()
print("Predicted future: ")
plt.imshow(decoded_future[TIMESTEP])
plt.show()
print("Actual RNN input: ")
for img in decoded_input_data:
    plt.imshow(img)
    plt.show()

In [ ]:
#How can I plot the current and predicted latent vectors side-
#by-side?
for timestep in range(len(generated_latent_vectors)):
    print("Current: ", rnn_input_data[timestep][-1])
    print("Predicted: ", generated_latent_vectors[timestep])